# house price prediction

In [1]:
import pandas as pd 
import torch 
import numpy as np 

In [5]:
df=pd.read_csv('datasets/houseprice.csv',usecols=['SalePrice','MSSubClass','MSZoning','LotFrontage','LotArea','Street','YearBuilt','LotShape','1stFlrSF','2ndFlrSF']).dropna()
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,YearBuilt,1stFlrSF,2ndFlrSF,SalePrice
0,60,RL,65.0,8450,Pave,Reg,2003,856,854,208500
1,20,RL,80.0,9600,Pave,Reg,1976,1262,0,181500
2,60,RL,68.0,11250,Pave,IR1,2001,920,866,223500
3,70,RL,60.0,9550,Pave,IR1,1915,961,756,140000
4,60,RL,84.0,14260,Pave,IR1,2000,1145,1053,250000


In [7]:
df.isna().sum()

MSSubClass     0
MSZoning       0
LotFrontage    0
LotArea        0
Street         0
LotShape       0
YearBuilt      0
1stFlrSF       0
2ndFlrSF       0
SalePrice      0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1201 entries, 0 to 1459
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MSSubClass   1201 non-null   int64  
 1   MSZoning     1201 non-null   object 
 2   LotFrontage  1201 non-null   float64
 3   LotArea      1201 non-null   int64  
 4   Street       1201 non-null   object 
 5   LotShape     1201 non-null   object 
 6   YearBuilt    1201 non-null   int64  
 7   1stFlrSF     1201 non-null   int64  
 8   2ndFlrSF     1201 non-null   int64  
 9   SalePrice    1201 non-null   int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 103.2+ KB


In [ ]:
# choose categorical features 
for i in df.columns:
    print(f'column name {i} are unique values are {len(df[i].unique())}')

column name MSSubClass are unique values are 15
column name MSZoning are unique values are 5
column name LotFrontage are unique values are 110
column name LotArea are unique values are 869
column name Street are unique values are 2
column name LotShape are unique values are 4
column name YearBuilt are unique values are 112
column name 1stFlrSF are unique values are 678
column name 2ndFlrSF are unique values are 368
column name SalePrice are unique values are 597


In [11]:
import datetime
datetime.datetime.now().year

2025

In [13]:
df['today year']=datetime.datetime.now().year-df['YearBuilt']
df.drop('YearBuilt',axis=1,inplace=True)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,1stFlrSF,2ndFlrSF,SalePrice,today year
0,60,RL,65.0,8450,Pave,Reg,856,854,208500,22
1,20,RL,80.0,9600,Pave,Reg,1262,0,181500,49
2,60,RL,68.0,11250,Pave,IR1,920,866,223500,24
3,70,RL,60.0,9550,Pave,IR1,961,756,140000,110
4,60,RL,84.0,14260,Pave,IR1,1145,1053,250000,25


In [14]:
# split between categorical and contineous features
cat_features=['MSSubClass','MSZoning','Street','LotShape']
out_feature='SalePrice' 

In [16]:
df['MSSubClass'].unique()

array([ 60,  20,  70,  50, 190,  45,  90, 120,  30,  80, 160,  75, 180,
        40,  85], dtype=int64)

In [18]:
from sklearn.preprocessing import LabelEncoder
lbl_encoders={}
lbl_encoders['MSSubClass']=LabelEncoder()
lbl_encoders['MSSubClass'].fit_transform(df['MSSubClass'])

array([5, 0, 5, ..., 6, 0, 0], dtype=int64)

In [20]:
# create a fucction for label encoding 
for feature in cat_features:
    lbl_encoders[feature]=LabelEncoder()
    df[feature]=lbl_encoders[feature].fit_transform(df[feature])

In [22]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,1stFlrSF,2ndFlrSF,SalePrice,today year
0,5,3,65.0,8450,1,3,856,854,208500,22
1,0,3,80.0,9600,1,3,1262,0,181500,49
2,5,3,68.0,11250,1,0,920,866,223500,24
3,6,3,60.0,9550,1,0,961,756,140000,110
4,5,3,84.0,14260,1,0,1145,1053,250000,25


In [23]:
# stackinh and coverting into tensors 
cat_features=np.stack([df['MSSubClass'],df['MSZoning'],df['Street'],df['LotShape']],1)
cat_features

array([[5, 3, 1, 3],
       [0, 3, 1, 3],
       [5, 3, 1, 0],
       ...,
       [6, 3, 1, 3],
       [0, 3, 1, 3],
       [0, 3, 1, 3]], dtype=int64)

In [24]:
# convert numpy to tensors 
import torch
cat_features=torch.tensor(cat_features,dtype=torch.int64)
cat_features

tensor([[5, 3, 1, 3],
        [0, 3, 1, 3],
        [5, 3, 1, 0],
        ...,
        [6, 3, 1, 3],
        [0, 3, 1, 3],
        [0, 3, 1, 3]])

In [27]:
# continous variables 
cont_features=[]
for i in df.columns:
    if i in ['MSSubClass','MSZoning','Street','LotShape','SalePrice']:
        pass 
    else:
        cont_features.append(i)
cont_features

['LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF', 'today year']

In [28]:
# stacking continuous varaibles to a tensor 
cont_values=np.stack([df[i].values for i in cont_features],axis=1)
cont_values=torch.tensor(cont_values,dtype=torch.float)
cont_values

tensor([[   65.,  8450.,   856.,   854.,    22.],
        [   80.,  9600.,  1262.,     0.,    49.],
        [   68., 11250.,   920.,   866.,    24.],
        ...,
        [   66.,  9042.,  1188.,  1152.,    84.],
        [   68.,  9717.,  1078.,     0.,    75.],
        [   75.,  9937.,  1256.,     0.,    60.]])

In [29]:
# dependent features 
y=torch.tensor(df['SalePrice'].values,dtype=torch.float).reshape(-1,1)
y

tensor([[208500.],
        [181500.],
        [223500.],
        ...,
        [266500.],
        [142125.],
        [147500.]])

In [31]:
cat_features.shape,cont_values.shape,y.shape

(torch.Size([1201, 4]), torch.Size([1201, 5]), torch.Size([1201, 1]))

# embeddings size for categorical columns 

In [ ]:
# embeddings layers to decide output based on input 
cat_dim=[len(df[col].unique()) for col in ['MSSubClass','MSZoning','Street','LotShape']]
cat_dim

[15, 5, 2, 4]

In [34]:
### thunb rule what dimension output shoud be set based on the input dimension 
embedding_dim=[(x,min(50,(x+1)//2)) for x in cat_dim]
embedding_dim

[(15, 8), (5, 3), (2, 1), (4, 2)]

In [35]:
# embedding layers to pytorch 
import torch 
import torch.nn as nn 
import torch.nn.functional as F 

embed_representation=nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dim])
embed_representation

ModuleList(
  (0): Embedding(15, 8)
  (1): Embedding(5, 3)
  (2): Embedding(2, 1)
  (3): Embedding(4, 2)
)

In [36]:
cat_features

tensor([[5, 3, 1, 3],
        [0, 3, 1, 3],
        [5, 3, 1, 0],
        ...,
        [6, 3, 1, 3],
        [0, 3, 1, 3],
        [0, 3, 1, 3]])

In [49]:
embedding_val=[]
for i,e, in enumerate(embed_representation):
    embedding_val.append(e(cat_features[:,i]))
embedding_val

[tensor([[-0.1974, -1.0191, -0.0540,  ..., -0.0909,  0.5276,  0.6296],
         [-0.3674,  2.0539, -0.0063,  ..., -0.5104, -0.4035, -0.9335],
         [-0.1974, -1.0191, -0.0540,  ..., -0.0909,  0.5276,  0.6296],
         ...,
         [-2.0306,  1.1506, -0.1909,  ...,  0.6665,  0.5868,  0.1768],
         [-0.3674,  2.0539, -0.0063,  ..., -0.5104, -0.4035, -0.9335],
         [-0.3674,  2.0539, -0.0063,  ..., -0.5104, -0.4035, -0.9335]],
        grad_fn=<EmbeddingBackward0>),
 tensor([[0.6532, 1.0872, 0.4690],
         [0.6532, 1.0872, 0.4690],
         [0.6532, 1.0872, 0.4690],
         ...,
         [0.6532, 1.0872, 0.4690],
         [0.6532, 1.0872, 0.4690],
         [0.6532, 1.0872, 0.4690]], grad_fn=<EmbeddingBackward0>),
 tensor([[1.2441],
         [1.2441],
         [1.2441],
         ...,
         [1.2441],
         [1.2441],
         [1.2441]], grad_fn=<EmbeddingBackward0>),
 tensor([[-1.0678,  0.3693],
         [-1.0678,  0.3693],
         [ 1.2488,  0.3335],
         ...,
   

In [50]:
z=torch.cat(embedding_val,1)
z 

tensor([[-0.1974, -1.0191, -0.0540,  ...,  1.2441, -1.0678,  0.3693],
        [-0.3674,  2.0539, -0.0063,  ...,  1.2441, -1.0678,  0.3693],
        [-0.1974, -1.0191, -0.0540,  ...,  1.2441,  1.2488,  0.3335],
        ...,
        [-2.0306,  1.1506, -0.1909,  ...,  1.2441, -1.0678,  0.3693],
        [-0.3674,  2.0539, -0.0063,  ...,  1.2441, -1.0678,  0.3693],
        [-0.3674,  2.0539, -0.0063,  ...,  1.2441, -1.0678,  0.3693]],
       grad_fn=<CatBackward0>)

In [52]:
# implement dropout 
dropout=nn.Dropout(0.4)
final_embed=dropout(z)
final_embed

tensor([[-0.0000, -1.6985, -0.0000,  ...,  0.0000, -1.7797,  0.6155],
        [-0.6123,  3.4231, -0.0000,  ...,  0.0000, -1.7797,  0.6155],
        [-0.3290, -1.6985, -0.0900,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.0000,  0.0000, -0.0000,  ...,  2.0735, -0.0000,  0.6155],
        [-0.6123,  3.4231, -0.0105,  ...,  2.0735, -0.0000,  0.6155],
        [-0.6123,  0.0000, -0.0105,  ...,  2.0735, -0.0000,  0.0000]],
       grad_fn=<MulBackward0>)

In [57]:
## create a neural network 
import torch
import torch.nn as nn 
import torch.nn.functional as F 

class FeedForwardNN(nn.Module):

    def __init__(self,embedding_dim,n_cont,out_sz,layers,p=0.5):
        super().__init__()
        self.embeds=nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dim])
        self.emb_drop=nn.Dropout(p)
        self.bn_cont=nn.BatchNorm1d(n_cont)

        layerlist=[]
        n_emb=sum((out for inp,out in embedding_dim))
        n_in=n_emb+n_cont 

        for i in layers:
            layerlist.append(nn.Linear(n_in,i))
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in=i 
        layerlist.append(nn.Linear(layers[-1],out_sz))
        self.layers=nn.Sequential(*layerlist)
    
    def forward(self,x_cat,x_cont):
        embeddings=[]
        for i,e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
        x=torch.cat(embeddings,1)
        x-self.emb_drop(x)
        x_cont=self.bn_cont(x_cont)
        x=torch.cat([x,x_cont],1)
        x=self.layers(x)
        return x 

In [58]:
len(cont_features)

5

In [60]:
torch.manual_seed(100)
model=FeedForwardNN(embedding_dim,len(cont_features),1,[100,50],p=0.4)
model

FeedForwardNN(
  (embeds): ModuleList(
    (0): Embedding(15, 8)
    (1): Embedding(5, 3)
    (2): Embedding(2, 1)
    (3): Embedding(4, 2)
  )
  (emb_drop): Dropout(p=0.4, inplace=False)
  (bn_cont): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=19, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [61]:
# define loss and optimizer
loss_function=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [62]:
df.shape

(1201, 10)

In [64]:
cont_values, cont_values.shape

(tensor([[   65.,  8450.,   856.,   854.,    22.],
         [   80.,  9600.,  1262.,     0.,    49.],
         [   68., 11250.,   920.,   866.,    24.],
         ...,
         [   66.,  9042.,  1188.,  1152.,    84.],
         [   68.,  9717.,  1078.,     0.,    75.],
         [   75.,  9937.,  1256.,     0.,    60.]]),
 torch.Size([1201, 5]))

In [ ]:
# train test split 